In [2]:
# Librería para manipular datasets
import pandas as pd
import numpy as np
import re

In [3]:
datos = pd.read_csv('./app/base_plataformas.csv', engine='python', decimal='.')
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Titulo        22998 non-null  object
 1   Duracion      22998 non-null  int64 
 2   Anio_estreno  22998 non-null  int64 
 3   Tipo          22998 non-null  object
 4   Genero        22998 non-null  object
 5   Actores       22998 non-null  object
 6   Plataforma    22998 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.2+ MB


Máxima duración según tipo de film (película/serie), por plataforma y por año: El request debe ser: get_max_duration(año, plataforma, [min o season])

In [9]:
def get_max_Duracion(Plataforma, año, tipo):
    if tipo == 'min':
        t = 'Movie'
        g = datos[datos['Plataforma'] == Plataforma][datos['Anio_estreno'] == año][datos['Tipo'] == t].groupby(['Plataforma', 'Tipo', 'Anio_estreno'])['Duracion'].idxmax()
        duracion = datos['Duracion'].get(g[0])
        titulo = datos['Titulo'].get(g[0])
        return (f'El Título es {titulo} y su duración es de {duracion} {tipo}')
    elif tipo == 'season':
        t = 'TV Show'
        g = datos[datos['Plataforma'] == Plataforma][datos['Anio_estreno'] == año][datos['Tipo'] == t].groupby(['Plataforma', 'Tipo', 'Anio_estreno'])['Duracion'].idxmax()
        duracion = datos['Duracion'].get(g[0]) 
        titulo = datos['Titulo'].get(g[0])
        return (f'El Título es {titulo} y su duración es de {duracion} {tipo}')

In [10]:
get_max_Duracion('Netflix',2017,'season')

C:\Users\emman\AppData\Local\Temp\ipykernel_8380\952170855.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  g = datos[datos['Plataforma'] == Plataforma][datos['Anio_estreno'] == año][datos['Tipo'] == t].groupby(['Plataforma', 'Tipo', 'Anio_estreno'])['Duracion'].idxmax()


'El Título es NCIS y su duración es de 15 season'

Cantidad de películas y series (separado) por plataforma El request debe ser: get_count_plataform(plataforma)

In [312]:
def get_count_plataform(plataforma):
    gcp = datos[datos['Plataforma'] == plataforma]
    gcp = gcp.groupby(['Plataforma'])['Tipo'].value_counts()
    gcp = pd.DataFrame(gcp)
    return gcp

In [313]:
get_count_plataform('Netflix')

Tipo
Plataforma Tipo         
Netflix    Movie    6131
           TV Show  2676

Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo. El request debe ser: get_listedin('genero')
Como ejemplo de género pueden usar 'comedy', el cuál deberia devolverles un cunt de 2099 para la plataforma de amazon.

In [11]:
def get_listedin (genero):
    gl = datos[datos.Genero.str.contains(genero, case=False)].groupby(by=['Plataforma']).Titulo.count()
    gl = pd.DataFrame(gl)
    gl.reset_index(inplace=True)
    gl.sort_values(by='Titulo', inplace=True, ascending=False)
    gl.reset_index(inplace=True, drop=True)
    return gl.iloc[0]

In [12]:
get_listedin ('comedy')

Plataforma    Amazon
Titulo          2099
Name: 0, dtype: object

Actor que más se repite según plataforma y año. El request debe ser: get_actor(plataforma, año)

In [7]:
def get_actor (plataforma,anio):
    actores_lista = []
    ga = datos.query(f"Plataforma == '{plataforma}' and Anio_estreno ==  {anio}").Actores.tolist()
    for i in range(len(ga)):
        actores_temp = ga[i].split(",")
        for j in range(len(actores_temp)):
            if actores_temp[j] != 'Sin dato':
                actores_lista.append(actores_temp[j])
    actores_dicc = dict(zip(actores_lista,map(lambda x: actores_lista.count(x),actores_lista)))
    actor_max = max(actores_dicc, key=actores_dicc.get)
    actor_max_aparece = actores_dicc.get(actor_max)
    return f'Es {actor_max} con {actor_max_aparece} apariciones.'

In [8]:
get_actor ("Netflix",2018)

'Es  Andrea Libman con 8 apariciones.'

In [ ]:
@app.get("/get_max_duration/{anio}/{plataforma}/{tipo}")
def get_max_duration(anio,plataforma,tipo):
    anio = int(anio)
    tuple_plataformas = ("amazon","disney","hulu","netflix")
    tuple_tipo = ("min","season")
    if (anio in range(df_final.release_year.min(),df_final.release_year.max()+1)) and (plataforma in (tuple_plataformas)) and (tipo in tuple_tipo):
        id_final = df_final.query(f"release_year == {anio} and platform == '{plataforma}' and duration_type == '{tipo}'").duration_int.idxmax()
        return f'{df_final[id_final:id_final+1].to_dict().get("title").get(id_final)} Tiene la mayor duración: {df_final[id_final:id_final+1].to_dict().get("duration").get(id_final)}'
    else:
        return f"Faltan datos para su consulta o alguno de los parámetros es incorrecto, por favor verifique que los datos ingresados."

@app.get("/get_count_platform/{plataforma}")
def get_count_platform(plataforma):
    tuple_plataformas = ("amazon","disney","hulu","netflix")
    if plataforma in (tuple_plataformas):
        dict = df_final.query(f"platform == '{plataforma}'").groupby(by=['type']).title.count().to_dict()
        return f'El conteo de la plataforma {plataforma} es: {dict.get("Movie")} películas y {dict.get("TV Show")} series'


@app.get("/get_listedin/{genero}")
def get_listedin(genero):
    conteo = df_final[df_final.listed_in.str.contains(genero, case=False)].groupby(by=['platform']).title.count().to_dict()
    return f'El género {genero} aparece {conteo.get("amazon")} veces en amazon, {conteo.get("disney")} veces en disney, {conteo.get("hulu")} veces en hulu y {conteo.get("netflix")} veces en netflix'


@app.get("/get_actor/{plataforma}/{anio}")
async def actores(plataforma,anio):
    anio = int(anio)
    tuple_plataformas = ("amazon","disney","hulu","netflix")
    if (anio in range(df_final.release_year.min(),df_final.release_year.max()+1)) and (plataforma in (tuple_plataformas)):
        actor_list = []
        cast_list = df_final.query(f"platform == '{plataforma}' and release_year == {anio}").cast.tolist()
        for i in range(len(cast_list)):
            actor_list_temp = cast_list[i].split(",")
            for j in range(len(actor_list_temp)):
                if actor_list_temp[j] != 'Sin Datos':
                    actor_list.append(actor_list_temp[j])
        dict_actor = dict(zip(actor_list,map(lambda x: actor_list.count(x),actor_list)))
        max_actor = max(dict_actor, key=dict_actor.get)
        max_actor_appearances = dict_actor.get(max_actor)
        return f'El actor/actriz con mayor número de apariciones en la plataforma {plataforma} el año {anio} es {max_actor} con {max_actor_appearances} apariciones.'
    else:
        return f"Faltan datos para su consulta o alguno de los parámetros es incorrecto, por favor verifique que los datos ingresados."